In [1]:
%pip install spacy nltk 

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 177, in handle_pip_version_check
    session = self._build_session(
              ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages

In [2]:
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize

In [3]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
import pandas as pd 
VLMresponses = pd.read_csv("VLM_responses_Baseline.csv")
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]
VLMresponses = VLMresponses.drop(["Why it Failed?", "Accurate"], axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt..."
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",..."
2,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ..."
3,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b..."
4,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\..."


In [6]:
# Sample Input
text = VLMresponses.iloc[22]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[22]["JSON"]
print(jsonForm)

To complete the task of logging in with current credentials, the user should follow these steps based on the visible UI elements:

1. Tap on the "Enter your SplashTop Account" button to enter the email field.
2. Type in the email associated with the SplashTop account.
3. Tap on the "Password" field to enter the password.
4. Type in the password associated with the SplashTop account.
5. Tap on the "Log in" button to log in with the entered credentials.
{
   "task":"login with my current credentials",
   "buttons":[
      "Enter your SplashTop Account",
      "Log in"
   ],
   "field":[
      "password",
      "email"
   ]
}


In [29]:
doc = nlp(text)

In [12]:
from spacy import displacy
displacy.render(doc, style="dep")

In [ ]:
def extract_task(doc):
    task = None
    for sent in doc.sents: # break down the document into sentences
        for token in sent: # look through every word inthe sentence
           if token.pos_ == "VERB" and token.dep_ in ("pobj", "pcomp"): # xcomp is the open clausal complement, pcomp is the prepositional complement, and ROOT is the main verb of the sentence.
                task = " ".join([t.text for t in token.subtree]) #  subtree includes the verb and all words attached to it in the tree.
               
        if task: # if a task has been found, break out of the loop
            return task
    return task

print(extract_task(doc))

logging in with current credentials


In [ ]:
def extract_buttons(doc):
    buttons = set()
    for sent in doc.sents:
        for token in sent:
            button_name = None
            #print(f"cur token: {token.lemma_}")
            if (token.lemma_).lower() in ("tap", "press", "click"):
                # print(f"Found action: {token.lemma_}")
                for child in token.children:
                    # print(f"Child of action: {child.text}")
                    if child.dep_ in ("prep"):
                        for grandchild in child.children:
                            # print(f"Grandchild of action: {grandchild.text} with dep {grandchild.dep_}")
                            for word in grandchild.subtree:
                                if word.dep_ in ("compound", "amod", "nmod", "pobj", "npadvmod"):
                                    print(f"Adding word to label: {word.text}")
                                    if button_name is None:
                                        button_name = word.text
                                    else:
                                        button_name += word.text
                                    button_name += " "  # Add space between words
                        buttons.add("".join(button_name.strip())) 
    return buttons

print(extract_buttons(doc))

Adding word to label: Enter
Adding word to label: SplashTop
Adding word to label: Account
Adding word to label: email
Adding word to label: Password
Adding word to label: field
Adding word to label: Log
Adding word to label: button
{'Log button', 'Enter SplashTop Account email', 'Password field'}
